In [7]:
import os
import json
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console

console = Console()

In [8]:
OPENAI_API_KEY = get_test_key()
# Set up your OpenAI API key
openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
console.print(OPENAI_API_KEY)

Success: .env file found with some environment variables

sk-proj-QDenE0JpMrlrM8CASrnAA9c-UkZh959UyFnxVIFK5CrvUGt3gDQ6PCPAelYd0GD7AKbUTpEx1zT3BlbkFJ-zaqpKBs5MNlixrDnd1bTAxm6
YK7z5216yESvF9lg6U9NGB3dEgNmsZAgxQNMcMgl3WW3wrOMA

In [9]:
def get_embedding(text, model="text-embedding-3-small"):

    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [10]:
conn = _conn_open()
cur = conn.cursor()

Conn: <connection object at 0x000002C078CA7DE0; dsn: 'user=postgres password=xxx dbname=postgres 
host=host.docker.internal', closed: 0>

In [11]:
console = Console()
FILENAME = "test_article_elements.json"
json_data = load_json(FILENAME)
print(f"Number of items: {len(json_data)}")
encoded_image_data = None
for i in range(len(json_data)):

    if json_data[i]["type"] == "Image":
        console.print(f"[dark_orange]---------Item Number IMAGE: {i + 1}--------[/]")
        el_image_b64 = json_data[i]["metadata"]["image_base64"]
        # console.print(el_image_b64[:40])
    if json_data[i]["type"] == "NarrativeText":

        console.print(f"[dark_orange]---------Item Number TEXT: {i + 1}--------[/]")
        el_text = json_data[i]["text"]
        el_embedding = get_embedding(el_text)

        # console.print(el_embedding)
        new_id = str(uuid4())
        new_element_id = randint(1000, 9999)
        sql = f"""
        INSERT INTO tbl_doc_elements (
            file_id, element_id, element_text, image_base64, embedding)
        VALUES 
        ('{new_id}','{new_element_id}','{el_text}',
        '{encoded_image_data}', ARRAY[{el_embedding}])
        RETURNING id;
        """

        try:
            cur.execute(sql)
            row = cur.fetchone()
            id = row[0]
            console.print(f"ID is {id}")
            conn.commit()
        except Exception as e:
            console.print(e)

Number of items: 3


---------Item Number IMAGE: 1--------

---------Item Number TEXT: 2--------

ID is 080b0b51-b288-4a76-bddf-6470ed240745

---------Item Number TEXT: 3--------

ID is 52096aa0-8aec-4893-a349-999d06a61f40

In [12]:
cur.close()
conn.close()